In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

### **Data Reading**

In [0]:
df=spark.read.format("parquet")\
   .load("abfss://bronze@databricksaniete.dfs.core.windows.net/products")

In [0]:
df.limit(10).display()

product_id,product_name,category,brand,price,_rescued_data
P0001,Clearly Its,Beauty,Nike,1868.54,null
P0002,Production Clear,Beauty,Apple,587.13,null
P0003,Culture Coach,Home,Revlon,1599.24,null
P0004,Movement Part,Sports,LG,651.71,null
P0005,Fact Name,Clothing,Samsung,1861.78,null
P0006,Usually Stop,Toys,Adidas,936.36,null
P0007,Reveal Current,Sports,Adidas,1954.02,null
P0008,Force Language,Beauty,Puma,1251.26,null
P0009,Stage Leg,Clothing,Samsung,1247.15,null
P0010,Leader Then,Sports,Sony,975.53,null


In [0]:
df=df.drop("_rescued_data")
df.createOrReplaceTempView("products")

### **Functions**

function using SQL

In [0]:
%sql
CREATE or REPLACE FUNCTION databricks_cata.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price*0.90

In [0]:
%sql
select product_id, price, round(databricks_cata.bronze.discount_func(price),2) as discounted_price
from products limit 10

product_id,price,discounted_price
P0001,1868.54,1681.69
P0002,587.13,528.42
P0003,1599.24,1439.32
P0004,651.71,586.54
P0005,1861.78,1675.6
P0006,936.36,842.72
P0007,1954.02,1758.62
P0008,1251.26,1126.13
P0009,1247.15,1122.44
P0010,975.53,877.98


**similarly if we want to use in dataframe directly**

In [0]:
df=df.withColumn("discounted_price",expr("databricks_cata.bronze.discount_func(price)"))
df.limit(10).display()

product_id,product_name,category,brand,price,discounted_price
P0001,Clearly Its,Beauty,Nike,1868.54,1681.686
P0002,Production Clear,Beauty,Apple,587.13,528.417
P0003,Culture Coach,Home,Revlon,1599.24,1439.316
P0004,Movement Part,Sports,LG,651.71,586.5390000000001
P0005,Fact Name,Clothing,Samsung,1861.78,1675.602
P0006,Usually Stop,Toys,Adidas,936.36,842.724
P0007,Reveal Current,Sports,Adidas,1954.02,1758.618
P0008,Force Language,Beauty,Puma,1251.26,1126.134
P0009,Stage Leg,Clothing,Samsung,1247.15,1122.4350000000002
P0010,Leader Then,Sports,Sony,975.53,877.977


function using PYTHON

In [0]:
%sql
CREATE or REPLACE FUNCTION databricks_cata.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
  return p_brand.upper()
$$

In [0]:
%sql
select product_id, brand, databricks_cata.bronze.upper_func(brand) as brand_upper
from products limit 10

product_id,brand,brand_upper
P0001,Nike,NIKE
P0002,Apple,APPLE
P0003,Revlon,REVLON
P0004,LG,LG
P0005,Samsung,SAMSUNG
P0006,Adidas,ADIDAS
P0007,Adidas,ADIDAS
P0008,Puma,PUMA
P0009,Samsung,SAMSUNG
P0010,Sony,SONY


### **DATA WRITING**

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@databricksaniete.dfs.core.windows.net/products")

In [0]:
%sql
CREATE TABLE if not EXISTS databricks_cata.silver.products_silver
USING DELTA
LOCATION 'abfss://silver@databricksaniete.dfs.core.windows.net/products'